This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [6]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = 'XXX'

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [1]:
# First, import the relevant modules
import requests
import numpy as np

In [9]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
url = 'https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json'
params = dict(api_key=API_KEY,start_date='2017-01-01',end_date='2017-12-31')
r = requests.get(url,params=params)
json_data = r.json()
print(json_data.keys())

dict_keys(['dataset_data'])


In [10]:
json_data['dataset_data'].keys()

dict_keys(['limit', 'transform', 'column_index', 'column_names', 'start_date', 'end_date', 'frequency', 'data', 'collapse', 'order'])

In [28]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
for k in json_data['dataset_data'].keys():
    print(k,type(json_data['dataset_data'][k]))

limit <class 'NoneType'>
transform <class 'NoneType'>
column_index <class 'NoneType'>
column_names <class 'list'>
start_date <class 'str'>
end_date <class 'str'>
frequency <class 'str'>
data <class 'list'>
collapse <class 'NoneType'>
order <class 'NoneType'>


In [31]:
print(json_data['dataset_data']['column_names'])
print('Frequency: ', json_data['dataset_data']['frequency'])
np.shape(json_data['dataset_data']['data'])

['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover']
Frequency:  daily


(255, 11)

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [75]:
# Calculate the highest and lowest opening prices
data = json_data['dataset_data']['data']
min_open = data[0][1]
max_open = data[0][1]
find_max = lambda x,y: x if y is None else np.max((x,y))
find_min = lambda x,y: x if y is None else np.min((x,y))
for i in range(1,len(data)):
    max_open = find_max(max_open,data[i][1])
    min_open = find_min(min_open,data[i][1])
print('Minimum Opening Price: ',min_open)
print('Maximum Opening Price: ',max_open)

Minimum Opening Price:  34.0
Maximum Opening Price:  53.11


In [79]:
# Largest change in one day
large_c = data[0][2]-data[0][3]
find_c = lambda x,y,z: x if (y is None)|(z is None) else np.max((x,y-z))
for i in range(1,len(data)):
    large_c = find_c(large_c,data[i][2],data[i][3])
print('Largest change over 1 day: ',large_c)

Largest change over 1 day:  2.8100000000000023


In [83]:
# Largest change between two days
large_c = np.abs(data[0][4]-data[1][4])
kind_c = np.sign(data[0][4]-data[1][4])
find_c = lambda x,y,z: x if ((y is None)|(z is None))else (y-z if (np.abs(y-z)>np.abs(x)) else x)
for i in range(1,len(data)-1):
    large_c = find_c(large_c,data[i][4],data[i+1][4])
print('Largest change form one day to another: ',large_c)
    

Largest change form one day to another:  -2.559999999999995


In [84]:
# Average trading volume
trad_vol = 0
ct = 0
for i in range(len(data)):
    if data[i][6] is not None:
        trad_vol += data[i][6]
        ct += 1
print('Average trading volume for 2017: ',trad_vol/ct)
        

Average trading volume for 2017:  89124.33725490196


In [85]:
# The Median trading volume
trad_vol = []
ct = 0
for i in range(len(data)):
    if data[i][6] is not None:
        trad_vol.append(data[i][6])
print('Median trading volume for 2017: ',np.median(trad_vol))

Median trading volume for 2017:  76286.0
